In [1]:
# Import libraries
from pathlib import Path
import acoular as ac
import numpy as np
import pyfar as pf
import time

from evaluation_config import eval_conf

In [2]:
initialTime = time.time()

#---------------------------------------------------------------------------------------------------------------
# initial definitions
# name of result file
resFileName = eval_conf["out_folder"] + "2025-01-28_15-59-01_400437_eval_test2"

# define microphone geometry
micgeofile = Path().resolve() / (eval_conf["in_folder"] + "/array_position_data/bassoon_cage_64_optimized.xml")
m = ac.MicGeom(from_file=micgeofile)

# define calculation grid
resolution = 0.05
# g = ac.RectGrid3D(x_min=0.35, x_max=1.15, y_min=0.15, y_max=1.15, z_min=0.25, z_max=1.65, increment = resolution)
# g = ac.RectGrid3D(x_min=0.25, x_max=1.25, y_min=0.10, y_max=1.25, z_min=0.15, z_max=1.75, increment = resolution)
g = ac.RectGrid3D(x_min=eval_conf["x_min"], x_max=eval_conf["x_max"], y_min=eval_conf["y_min"], y_max=eval_conf["y_max"], z_min=eval_conf["z_min"], z_max=eval_conf["z_max"], increment=eval_conf["calc_grid_res_meters"])

# define audio data
name = eval_conf["in_folder"] + "/array_audio_data/2025-01-28_15-59-01_400437.h5"
data = ac.MaskedTimeSamples(name = name)

# define time related variables
start = 0#83#26        # in s
stop = 105#35#92#36
framerate = eval_conf["framereate_fps"]         # in f/s

# define freqency resolution
bandwith = eval_conf["bandwidth"]            # Octave = 1; 1/3 Octave = 3
frequency_bands = eval_conf["frequency_bands"] # Array with mid frequencies of frequency bands

#---------------------------------------------------------------------------------------------------------------
# calculation of fixed values
# st = ac.SteeringVector(grid=g, mics=m, steer_type='true location')
frameLength = 1/framerate

print('init time: ' + str(time.time()-initialTime) + ' s')

result = np.zeros(
    [len(frequency_bands),
     int((stop-start)*framerate),
     g.nxsteps,
     g.nysteps,
     g.nzsteps]
     )


init time: 0.01734018325805664 s


In [3]:
# begin evaluation loop

for i, currentFreqBand in enumerate(frequency_bands):

    for frame in range(0, int((stop-start)*framerate)):
        calcTime = time.time()

        print('Frame ' + str(frame+1)+' of '+str((stop-start)*framerate))
        print('Frequency Band: ' + str(i+1) + ' (' + str(currentFreqBand) + ' Hz)')
        print('Frame length: ' + str((start+(frameLength*(frame+1)))-(start+(frameLength*frame))) + ' s ')
        

        # data = ac.MaskedTimeSamples(name = name)
        data = ac.MaskedTimeSamples(name = name, start = (start+(frameLength*frame))*data.sample_freq, stop = (start+(frameLength*(frame+1)))*data.sample_freq)
        # print('length of cropped measurement: ' + str(data.numsamples/data.sample_freq) + ' s (' + str(data.numsamples) + ' samples)')

        f = ac.PowerSpectra(source=data, window='Hanning', overlap=eval_conf["fft_overlap"], block_size=eval_conf["fft_block_size"])
        st = ac.SteeringVector(grid=g, mics=m, steer_type='true location')
        b = ac.BeamformerCleansc(freq_data=f, steer=st)

        result[i,frame] = b.synthetic(currentFreqBand, bandwith)

        print('Calculation time:', np.round(time.time()-calcTime, 2), "seconds")
        print("")


Frame 1 of 10.5
Frequency Band: 1 (125 Hz)
Frame length: 10.0 s 
[('2025-01-28_15-59-01_400437_cache.h5', 1)]
Calculation time: 0.14 seconds

Frame 2 of 10.5
Frequency Band: 1 (125 Hz)
Frame length: 10.0 s 
[('2025-01-28_15-59-01_400437_cache.h5', 2)]
Calculation time: 0.12 seconds

Frame 3 of 10.5
Frequency Band: 1 (125 Hz)
Frame length: 10.0 s 
[('2025-01-28_15-59-01_400437_cache.h5', 3)]
Calculation time: 0.13 seconds

Frame 4 of 10.5
Frequency Band: 1 (125 Hz)
Frame length: 10.0 s 
[('2025-01-28_15-59-01_400437_cache.h5', 4)]
Calculation time: 0.12 seconds

Frame 5 of 10.5
Frequency Band: 1 (125 Hz)
Frame length: 10.0 s 
[('2025-01-28_15-59-01_400437_cache.h5', 5)]
Calculation time: 0.12 seconds

Frame 6 of 10.5
Frequency Band: 1 (125 Hz)
Frame length: 10.0 s 
[('2025-01-28_15-59-01_400437_cache.h5', 6)]
Calculation time: 0.12 seconds

Frame 7 of 10.5
Frequency Band: 1 (125 Hz)
Frame length: 10.0 s 
[('2025-01-28_15-59-01_400437_cache.h5', 7)]
Calculation time: 0.12 seconds

Frame 

In [4]:
# save Data
np.save(resFileName,result)
pf.io.write(resFileName, 
           start = start, 
           stop = stop, 
           framerate = framerate, 
           frames = int((stop-start)*framerate), 
           frequency_bands = frequency_bands, 
           bandwith = bandwith,
           resolution = resolution,
           name = name)

In [5]:
# read Data

result = np.load(resFileName + ".npy")
start, stop, framerate, frames, frequency_bands, bandwith, resolution, name = pf.io.read(resFileName+".far")
result_Lp = ac.L_p(result)        # convert results in sound pressure Level